# Обзор модуля `astropy.table`

Модуль `astropy.table` предоставляет гибкую среду для работы со структуированными астрономическими таблицами. В отличие от `pandas`, широко используемом при анализе данных, `astropy.table` специально разработан под нужды астрономии и бесшовно интегрируется с остальной экосистемой `astropy`.

Основная структура `astropy.table.Table`, в отличие от `pandas.DataFrame`, оптимизирована под научные нужды:

- Встроенная поддержка единиц измерения `astropy.units`
- Родная поддержка астрономических форматов данных **FITS**, **ECSV**, **VOTable**, и других
- Простое использование колоночных метаданных (единицы измерения, описания, форматы, маски)
- Полная совместимость с `SkyCoord`, `Time`, и другими объектами Astropy
- Точный контроль формата ввода и вывода данных
- Интеграция с `astroquery`, `astropy.io` и графопостроителями

Тем самым `astropy.table` подчеркивает научную прозрачность, воспроизводимость и учет единиц измерения, — все необходимое для надежных астрономических вычислений.

---

## Место в экосистеме Astropy

Также, как `astropy.units` необходим для надежности в размерных вычислениях, `astropy.table` необходим для обработки, организации и анализа астрономических датасетов. В Astropy он появляется:

- Как выходной формат запросов из онлайн-каталогов (например, через `astroquery`)
- Как результат чтения данных типа **FITS** и **ASCII**
- При манипуляциях с журналами наблюдений и результатами симуляций
- Как структура данных, передаваемая в функции построения графиков и моделей

---

В этом ноутбуке разберем структуру, создание, преобразование и возможности ввода и вывода у `astropy.table.Table`.

In [ ]:
from astropy.table import QTable, Table
import astropy.units as u
import numpy as np

star_names = np.array(["Proxima Centauri", "Alpha Centauri A", "Barnard's Star", "Wolf 359", "Luyten 726-8A"])
distances = np.array([4.24, 4.37, 5.96, 7.86, 8.73]) * u.lyr
apparent_mags = np.array([11.13, -0.01, 9.54, 13.5, 12.6]) * u.mag
absolute_mags = np.array([15.5, 4.38, 13.2, 16.6, 15.0]) * u.mag
temperatures = np.array([3042, 5790, 3134, 2800, 2650]) * u.K
radii = np.array([0.14, 1.22, 0.18, 0.16, 0.12]) * u.R_sun
spectral_types = np.array(["M5.5Ve", "G2V", "M4.0Ve", "M6.0Ve", "M5.5Ve"])

stars_table = QTable(
    [star_names, distances, apparent_mags, absolute_mags, temperatures, radii, spectral_types],
    names=["Star", "Distance", "App_Mag", "Abs_Mag", "Temperature", "Radius", "Spectral_Type"])

stars_table

### Описание создания таблиц

В ячейке выше мы создали таблицу с поддерживаемыми единицами измерения `QTable` из массивов Numpy:

- Переменная `star_names` содержит имена 5 ближайших звезд (в виде списка строк).
- `distances` содержит их расстояние от Земли в световых годах, с единицами из `astropy.units`.
- `apparent_mags` и `absolute_mags` содержат видимую и абсолютную звездную величину звезд.
- `temperatures` содержит эффективную температуру в кельвинах.
- `radii` содержит радиусы звезд в радиусах Солнца (`R_sun`).
- `spectral_types` содержит спектральные классы звезд.

In [ ]:
print(stars_table) # print() выводит ascii-форматированную таблицу с заголовком из имени и единицы измерения

In [ ]:
stars_table.colnames # список из названий колонок

In [ ]:
stars_table.info # общая информация про колонки в таблице

In [ ]:
stars_table['Temperature'].info.format = '5.0f' # с помощью .info.format можно задать формат вывода таблицы через print и сохранения в файл
print(stars_table)

In [ ]:
stars_table.info # добавился format!

In [ ]:
print(type(stars_table['Temperature']))

In [ ]:
print(stars_table['Temperature']) # вывести колонку
print(stars_table['Temperature'][0]) # вывести первую строку колонки
print(stars_table[0]['Temperature']) # вывести колонку Temperature у первой строки

In [ ]:
print(stars_table['Temperature'].value)   # вытащить значение
print(type(stars_table['Temperature'].value))

И `stars_table['col'][row]`, и `stars_table[row]['col']` позволяют обратиться к конкретной ячейке, однако вторая форма (`table[row]['col']`) полезна для доступа к нескольким колонкам одной строки через срезы:

In [ ]:
print(stars_table[0:2])

In [ ]:
print(stars_table['Temperature','Distance']) 

In [ ]:
print(stars_table['Temperature'].unit)

### Создание новой колонки

Сосчитаем новую колонку по закону Стефана-Больцмана:

\\[
L = 4\pi R^2 \sigma T^4
\\]

Пользуясь `astropy.constants` и `astropy.units`, мы гарантируем сохранность единиц измерения. Это не только упрощает вычисления, но и убирает ошибки размерностей.

In [ ]:
from astropy.constants import sigma_sb

R = stars_table['Radius'] 
T = stars_table['Temperature']

luminosity = 4 * np.pi * R**2 * sigma_sb * T**4

После подсчитывания, переведем светимости в солнечные и запишем в колонку `Luminosity` в таблице `stars_table`:

In [ ]:
luminosities_Lsun = luminosity.to(u.L_sun)

stars_table['Luminosity'] = luminosities_Lsun  # добавляем новую колонку
stars_table

Чтобы убрать лишние знаки после запятой, зададим форматирование новой колонки:

In [ ]:
stars_table['Luminosity'].info.format = '5.4f'
stars_table

Именно такое же форматирование сохранится, если записать таблицу в файл с помощью метода `write("filename")`:

In [ ]:
stars_table.write("test_table_4.csv")
stars_table['Luminosity'].info.format = '5.5f'
stars_table.write("test_table_5.csv")

Чтение сохраненных таблиц в большинстве распространенных форматов осуществляется методом `read`:

In [ ]:
stars = QTable.read("test_table_4.csv")
stars

In [ ]:
# здесь можно прочитать про все доступные для чтения/записи форматы
QTable.read.help()

## Общие принципы редактирования таблиц

Создадим таблицу из четырех колонок разных типов и посмотрим на её редактирование. Общие принципы совпадают с такими же в `pandas` и `numpy`:

In [ ]:
a = np.array([1, 4, 5], dtype=np.int32)
b = [2.0, 5.0, 8.5]
c = ['x', 'y', 'z']
d = [10, 20, 30] * u.m / u.s

t = QTable([a, b, c, d],
           names=('a', 'b', 'c', 'd'),
           meta={'name': 'first table'})
t

In [ ]:
t['a'][:] = [-1, -2, -3]    # задать значения "на месте" (должен быть тот же самый тип!)
t['a'][2] = 30               # задать строку 2 колонки 'a'
t[1] = (8, 9.0, "W", 4 * u.m / u.s) # задать все значения строки 1
t[1]['b'] = -9              # задать колонку 'b' строки 1
t[0:2]['b'] = 100.0         # задать колонки 'b' строк 0 и 1
t

Колонки можно заменять, добавлять, удалять и переименовывать следующим образом:

In [ ]:
t['b'] = ['a', 'new', 'dtype']   # заменить колонку 'b' (можно на другой тип)
t['e'] = [1, 2, 3]               # добавить колонку 'e'
del t['c']                       # удалить колонку 'c'
t.rename_column('a', 'A')        # Переименовать колонку 'a' в 'A'
t.colnames

In [ ]:
t

Новую строку можно добавлять вот так. Обратите внимание, что 10 см/c автоматически переведутся в нужные единицы (м/с)

In [ ]:
t.add_row([-8, 'string', 10 * u.cm / u.s, 10])
t['d']

## Отличия Table и QTable

Каталоги ниже будут загружены в виде типа данных `astropy.table.Table`, который практически идентичен `QTable`. Исключение заключается в том, что колонки в `Table` являются специфическими массивами типа `Table.columns`, а не типом `Quantity`. Это сделано для того, чтобы обращение к конкретной ячейке возвращало безразмерный объект. Из-за этого при некоторых размерных преобразованиях может возникнуть путаница в единицах. В случае сомнений при работе с размерными единицами в `Table`, добавьте `.quantity` в название колонки.

In [ ]:
data = [[30, 90]]

t = Table(data, names=('angle',))
t['angle'].unit = 'deg'
np.sin(t['angle']) # синус воспринял градусы в радианах, несмотря на единицу

In [ ]:
np.sin(t['angle'].quantity)

In [ ]:
t = QTable(data, names=('angle',))
t['angle'].unit = 'deg'
np.sin(t['angle']) # с QTable каждая колонка уже Quantity

## Загрузка данных с помощью `astroquery` и `Vizier`

Библиотека `astroquery` позволяет обращаться к таким базам астрономических данных, как [SIMBAD](http://simbad.cds.unistra.fr/simbad/) и [VizieR](https://vizier.cds.unistra.fr/viz-bin/VizieR), и загружать информацию о конкретном объекте, или данные каталога целиком, или (что более актуально для огромных каталогов) конкретную область каталога, заданную по координатам её центра. Полученный объект имеет тип `astropy.table.Table` и обрабатывается аналогично процедуре выше.

In [ ]:
from astroquery.vizier import Vizier
import astropy.coordinates as coord
import astropy.units as u
help(Vizier) # большая справка по методу. Где-то может обманывать, требуется эксперимент

In [ ]:
# найдем с помощью объекта SkyCoord координаты Сириуса в ICRS по имени
alphaCMa = coord.SkyCoord.from_name("Sirius")
alphaCMa

In [ ]:
# так можно поискать каталоги по ключевым словам через пробел
catalog_list = Vizier.find_catalogs('Variable Catalogue')
for k, v in catalog_list.items():
    print(k, ":", v.description)

In [ ]:
# Загрузим Общий Каталог Переменных Звезд (ОКПЗ) 2017 года
result = Vizier().get_catalogs(catalog="B/gcvs")
result[0]

## **!ВНИМАНИЕ!** ячейка ниже загружает порядка ~10 МБ табличных данных

In [ ]:
# Загрузим ВЕСЬ Общий Каталог Переменных Звезд (ОКПЗ) 2017 года
result = Vizier(row_limit=-1,).get_catalogs(catalog="B/gcvs")
result[0]

In [ ]:
# Загрузим переменные звезды из каталога ОКПЗ B/gcvs возле Сириуса в квадратной области 5x5 градусов с периодами меньше 100 дней
result = Vizier(row_limit=-1, column_filters={"Period": "< 100"}).query_region(coordinates=alphaCMa, width="5deg", catalog="B/gcvs")
result[0] # смотрим на первую и единственную таблицу в списке таблиц

Мы передали в качестве аргументов при инициализации класса Vizier `row_limit=-1` для того, чтобы убрать ограничения по количеству загружаемых строк (по-умолчанию их загружается не более 50), и `column_filters` - словарь, опциональный аргумент, задает фильтр на колонки, содержит название колонки как ключ, и ограничивающее условие как значение.

Метод `.get_catalogs(catalog='X')` позволяет полностью загрузить каталог, в нашем случае `X` — это `B/gcvs`

Метод `.query_region` позволяет загрузить область круглой (кольцевой) или квадратной (прямоугольной) формы и имеет следующие параметры:
- `coordinates`: объект типа str или `astropy.coordinates`, центр области в угловых единицах или приводимым к ним.
- `radius`: преобразуемый в `~astropy.coordinates.Angle` объект, (внешний) радиус выделяемой области
- `inner_radius`: преобразуемый в `~astropy.coordinates.Angle` объект, указанный вместе с ``radius`` превращает выделенную область в кольцо с
внешним радиусом ``radius`` и внутренним радиусом ``inner_radius``.
- `width` : преобразуемый в `~astropy.coordinates.Angle` объект, ширина стороны прямоугольной области, если задана без `height`, область будет квадратной.
- `height` : преобразуемый в `~astropy.coordinates.Angle` объект, указанный вместе с ``width``, задает выделяемую область прямоугольной с шириной ``width`` и высотой ``height``.
- `catalog`: аналогично предыдущему методу, указывает конкретный каталог для загрузки, в нашем случае `"B/gcvs"`.

In [ ]:
import matplotlib.pyplot as plt
from astropy.visualization import quantity_support
quantity_support()

coo = coord.SkyCoord(result[0]['RAJ2000'], result[0]['DEJ2000'], frame='icrs', unit=(u.hourangle, u.deg)) # считываем согласно формату каталога
fig, ax = plt.subplots()
ax.scatter(coo.ra, coo.dec)
ax.scatter(alphaCMa.ra, alphaCMa.dec, c='yellow', s=50)
ax.grid(alpha=0.2)
ax.set_title("Переменные в области Сириуса")
ax.set_xlabel("α(ICRS), градусы")
ax.set_ylabel("δ(ICRS), градусы")

In [ ]:
print(result[0]['Period']) # обратите внимание - колонка имеет специфический тип Column или MaskedColumn
print(type(result[0]['Period']))
print(result[0]['Period'].quantity) # таким методом можно перевести размерную величину в массив Quantity

In [ ]:
result[0]['Period'].format = '.5f' # форматирование работает аналогично
print(result[0]['Period'])

## Упражнения

### 0. Точка отсчета
- Создайте момент времени типа `astropy.time.Time` с датой вашего рождения и временем, соответствующим полуночи по UTC: например, `"10-04-1998 00:00:00"`, запишите его в переменную `b_time`.
- Создайте объект типа `EarthLocation` с местом вашего рождения , запишите его в переменную `b_loc`. (Геодезические широту и долготу места можно взять, например, из [maps.yandex.ru])
- Создайте объект типа `SkyCoord`, в который запишите координаты ICRS местного астрономического зенита в момент `b_time` в месте `b_loc`, запишите его в переменную `b_zenith`. Выведите координаты `b_zenith`.

### 1. Переменный зенит

- Найдите с помощью `Vizier.find_catalogs` международный каталог переменных звезд `VSX`.
- Загрузите из этого каталога с помощью `Vizier.query_region` все звезды в квадратной области 2°×2° с центром в `b_zenith` в таблицу `var_table`.
- С помощью `seaborn.countplot` постройте столбчатую диаграмму с количеством переменных звезд разных типов в таблице `var_table` (используйте метод `to_pandas`, чтобы преобразовать `Table` в `DataFrame`). Сделайте диаграмму эстетически красивой.
  
### 2. Астрометрический зенит
- Загрузите из оптического каталога Gaia DR3 `I/355/gaiadr3` круглую область радиусом 10' с центром в `b_zenith` из предыдущего задания в таблицу `gaia_table`.
- Выведите количество звезд в таблице `gaia_table`.
- Постройте ГР-диаграмму в виде диаграммы рассеяния в координатах `BP-RP` и `Gmag`. Сделайте её эстетически красивой. Не забудьте инвертировать ось звездной величины.
  
### 3. Инфракрасное небо (с этой задачей беда!)
- Загрузите из инфракрасного каталога 2MASS `VII/233/xsc` *все звезды* со звездной величиной `J.ext` ярче 10 и запишите в таблицу `twomass_table` 
- Выведите количество звезд в таблице `twomass_table`
- Создайте две новых колонки с галактическими координатами (l, b) каждого источника.
- Постройте диаграмму рассеяния источников из `twomass_table` в галактических координатах. Сделайте её эстетически красивой. Добейтесь того, чтобы центр Галактики находился в центре диаграммы рассеяния.
- (вопрос на дополнительное изучение) Что вы можете сказать о ярких в инфракрасных лучах структурах?
